In [1]:
import os
import json

# List result Files
experiment_files = [a for a in os.listdir("./out") if ".json" in a]

# Load result Files
all_experiments = []
for filename in experiment_files:
    with open("./out/{}".format(filename), "r") as f:
        all_experiments.append(json.load(f))

# Aglomerate similar results
unique_experiment_names = list(dict.fromkeys([a["model"] for a in all_experiments]))
print(unique_experiment_names)

sorted_experiments = {}
for e in unique_experiment_names:
    sorted_experiments[e] = []

for e in all_experiments:
    sorted_experiments[e['model']].append(e)

['lee2021_layer_7', 'matsubara2022_1', 'ours_size_34', 'matsubara2022_4', 'ours_size_24', 'ours_size_44', 'lee2021_layer_3', 'ours_size_11', 'dummy']


In [2]:
baseline = sorted_experiments['dummy']

total_energy = sum([b['joules1'] for b in baseline])
total_seconds = sum([b['seconds'] for b in baseline])
baseline_power = total_energy/total_seconds
print(baseline_power)

3.5019126621000005


In [3]:
aggregated_experiments = []

for exp_name, exp_list in sorted_experiments.items():
    joules = sum([e['joules1'] for e in exp_list])
    seconds = sum([e['seconds'] for e in exp_list])
    corrected_joules = joules - baseline_power*seconds
    num_images = sum([e['num_images'] for e in exp_list])
    aggregated_experiments.append({
        "joules": joules,
        "seconds": seconds,
        "corrected_joules": joules - baseline_power*seconds,
        "corrected_joules_per_image": corrected_joules / num_images,
        "inference_time": seconds / num_images,
        "num_images": num_images,
        "model": exp_name,
        "macs(MMAC)": exp_list[0]['macs']/1e6,
        "bw(KB)": exp_list[0]['bw']/1e6,
        "map": exp_list[0]['map'],
        "params": exp_list[0]['params'],
    })
    
with open("test.json", "w+") as f:
    json.dump(aggregated_experiments, f)

In [4]:
import pandas as pd
df = pd.read_json('test.json')
df = df.set_index("model")
df = df.sort_index(ascending=True)
print(df)

                     joules  seconds  corrected_joules  \
model                                                    
dummy             70.038253       20          0.000000   
lee2021_layer_3  186.238937       40         46.162431   
lee2021_layer_7   92.441149       20         22.402896   
matsubara2022_1  196.346035       40         56.269528   
matsubara2022_4  286.732491       60         76.617731   
ours_size_11      85.969183       20         15.930930   
ours_size_24     177.742410       40         37.665904   
ours_size_34      90.266119       20         20.227866   
ours_size_44     183.065250       40         42.988743   

                 corrected_joules_per_image  inference_time  num_images  \
model                                                                     
dummy                              0.000000        0.588235          34   
lee2021_layer_3                    0.369299        0.320000         125   
lee2021_layer_7                    0.546412        0.487805  

In [5]:
print(df.to_latex(index=False)) 

\begin{tabular}{rrrrrrrrrr}
\toprule
    joules &  seconds &  corrected\_joules &  corrected\_joules\_per\_image &  inference\_time &  num\_images &  macs(MMAC) &   bw(KB) &  map &  params \\
 70.038253 &       20 &          0.000000 &                    0.000000 &        0.588235 &          34 &    0.000000 & 0.000000 &  0.0 &       0 \\
\midrule
186.238937 &       40 &         46.162431 &                    0.369299 &        0.320000 &         125 &  541.491200 & 0.030500 & 36.8 &   29792 \\
 92.441149 &       20 &         22.402896 &                    0.546412 &        0.487805 &          41 & 1282.252800 & 0.008800 & 36.5 &  595680 \\
196.346035 &       40 &         56.269528 &                    2.084057 &        1.481481 &          27 & 2897.102848 & 0.180000 & 36.1 &   63264 \\
286.732491 &       60 &         76.617731 &                    1.915443 &        1.500000 &          40 & 2897.102848 & 0.015000 & 29.5 &   63264 \\
 85.969183 &       20 &         15.930930 &           

/tmp/ipykernel_1308/1261223815.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))
